# BIRD identification model

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from keras.preprocessing import image  

C:\Users\kalai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [5]:
IMAGE_RES = 224                                                                 # input dimensions required by the CNN model
def preprocess_image_to_tensor(img_path, ax):
    img = image.load_img(img_path, target_size=(IMAGE_RES, IMAGE_RES))          # loads RGB image as PIL.Image.Image type, resize image to model input dimensions
    ax.imshow(img, alpha=1)
    x = image.img_to_array(img)/255.0                                           # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3) and normalize to [0:1] as per model requierements
    x = np.expand_dims(x, axis=0)                                               # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return x                                                                    # return 4D tensor

In [6]:
URL = 'https://tfhub.dev/google/aiy/vision/classifier/birds_V1/1'               # Import pre-trained bird classification model from Tensorflow Hub 
bird = hub.KerasLayer(URL, input_shape=(IMAGE_RES,IMAGE_RES,3))                 # Using aiy/vision/classifier/birds_V1 classifying 964 bird species from images. It is based on MobileNet, and trained on photos contributed by the iNaturalist community
bird.trainable=False                                                            # freeze model weights

In [7]:
model=tf.keras.Sequential([bird])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 965)               0         
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


## Load labels

In [8]:
#from google.colab import drive
#drive.mount('/content/drive')
#!ls "/content/drive/My Drive/Colab Notebooks"

In [10]:
#labels = pd.read_csv("/content/drive/My Drive/Colab Notebooks/aiy_birds_V1_labelmap.csv", sep=',', header=0, index_col=0)  # using scientific names  in Latin
labels = pd.read_csv("labels_oiseaux.csv", sep=';', header=0, index_col=0)          # file providing species in french, english and latin
labels.head()

,nom,name
id,,
964,Pas d'oiseau identifié,background
0,Roselin de Cassin,Cassin's Finch
1,Courlan brun,Limpkin
2,Rupornis magnirostris,Rupornis magnirostris
3,Geai bleu,Blue Jay


In [ ]:
lang='fr'                                                                       # select 'fr' for species in french, 'en' for english labels
if lang=='fr':
  label='nom'
else:
  label='name'
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
df = pd.read_excel(url)

# Preprocess the data
df = df.dropna()  # Remove rows with missing values
df = df[['Quantity', 'UnitPrice', 'CustomerID']]  # Select relevant features
df['IsBird'] = df['CustomerID'].apply(lambda x: 1 if x % 2 == 0 else 0)  # Create a binary target variable (IsBird)

# Split the data into features (X) and target variable (y)
X = df[['Quantity', 'UnitPrice', 'CustomerID']]
y = df['IsBird']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
my_files = np.array(glob("/content/drive/My Drive/Colab Notebooks/images/*"))   
num_img=my_files.shape[0]
lines= num_img//5 + 1
columns=np.minimum(5,num_img)
plt.figure(figsize=(30,15))
for index, file in enumerate(my_files):
  ax=plt.subplot(lines, columns,index+1)
  processed_image=preprocess_image_to_tensor(file, ax)                        
  output = model.predict(processed_image)                                     
  prediction = np.argmax(tf.squeeze(output).numpy())
 
  plt.title(labels[label][prediction])
plt.show()


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
df = pd.read_excel(url)

# Preprocess the data
df = df.dropna()  # Remove rows with missing values
df = df[['Quantity', 'UnitPrice', 'CustomerID']]  # Select relevant features
df['IsBird'] = df['CustomerID'].apply(lambda x: 1 if x % 2 == 0 else 0)  # Create a binary target variable (IsBird)

# Split the data into features (X) and target variable (y)
X = df[['Quantity', 'UnitPrice', 'CustomerID']]
y = df['IsBird']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8783398471105868
